In [48]:
%reload_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('/home/yuncong/Brain/pipeline_scripts')
from utilities2014 import *
import os

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2


os.environ['GORDON_DATA_DIR'] = '/home/yuncong/project/DavidData2014tif/'
os.environ['GORDON_REPO_DIR'] = '/home/yuncong/Brain'
os.environ['GORDON_RESULT_DIR'] = '/home/yuncong/project/DavidData2014results/'
os.environ['GORDON_LABELING_DIR'] = '/home/yuncong/project/DavidData2014labelings/'

dm = DataManager(data_dir=os.environ['GORDON_DATA_DIR'], 
  repo_dir=os.environ['GORDON_REPO_DIR'], 
  result_dir=os.environ['GORDON_RESULT_DIR'], 
  labeling_dir=os.environ['GORDON_LABELING_DIR'])

class args:
    stack_name = 'RS141'
    resolution = 'x5'
    slice_ind = 4
    gabor_params_id = 'blueNisslWide'
    segm_params_id = 'blueNisslRegular'
    vq_params_id = 'blueNissl'
    
dm.set_image(args.stack_name, args.resolution, args.slice_ind)
dm.set_gabor_params(gabor_params_id=args.gabor_params_id)
dm.set_segmentation_params(segm_params_id=args.segm_params_id)
dm.set_vq_params(vq_params_id=args.vq_params_id)

In [2]:
from joblib import Parallel, delayed

In [3]:
# Over-segment the image into superpixels using SLIC (http://ivrg.epfl.ch/research/superpixels)

from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_ubyte, pad

masked_image = dm.image.copy()

# approx_bg_intensity = dm.image[10:20, 10:20].mean()
# masked_image[~dm.mask] = approx_bg_intensity

masked_image[~dm.mask] = 0

segmentation = slic(gray2rgb(masked_image), n_segments=int(dm.segm_params['n_superpixels']), 
                    max_iter=10, 
                    compactness=float(dm.segm_params['slic_compactness']), 
                    sigma=float(dm.segm_params['slic_sigma']), 
                    enforce_connectivity=True)

In [ ]:
# display(mark_boundaries(masked_image, segmentation))

In [4]:
n = len(np.unique(segmentation))

def f(i):
    m = dm.mask[segmentation == i]
    return np.count_nonzero(m)/float(len(m)) < .8

q = Parallel(n_jobs=16)(delayed(f)(i) for i in range(n))
                        
masked_segmentation = segmentation.copy()
for i in np.where(q)[0]:
    masked_segmentation[segmentation == i] = -1

In [ ]:
# display(mark_boundaries(masked_image, masked_segmentation))

In [5]:
from skimage.segmentation import relabel_sequential

# try:
#     masked_segmentation_relabeled = dm.load_pipeline_result('segmentation', 'npy')

# except:

    # segmentation starts from 0
masked_segmentation_relabeled, fw, inv = relabel_sequential(masked_segmentation + 1)

# make background label -1
masked_segmentation_relabeled -= 1


In [6]:
dm.save_pipeline_result(masked_segmentation_relabeled, 'segmentation', 'npy')

saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_segm-blueNisslRegular_segmentation.npy


In [55]:
# masked_segmentation_relabeled = dm.load_pipeline_result('segmentation', 'npy')

In [7]:
from joblib import Parallel, delayed

sp_props = regionprops(masked_segmentation_relabeled + 1, intensity_image=dm.image, cache=True)

def obtain_props_worker(i):
    return sp_props[i].centroid, sp_props[i].area, sp_props[i].mean_intensity, sp_props[i].bbox

r = Parallel(n_jobs=16)(delayed(obtain_props_worker)(i) for i in range(len(sp_props)))

sp_centroids = np.array([s[0] for s in r])
sp_areas = np.array([s[1] for s in r])
sp_mean_intensity = np.array([s[2] for s in r])
sp_bbox = np.array([s[3] for s in r]) 

# n_superpixel x 8: (cy, cx, area, mean_intensity, ymin, xmin, ymax, xmax)

sp_properties = np.column_stack([sp_centroids, sp_areas, sp_mean_intensity, sp_bbox])

dm.save_pipeline_result(sp_properties, 'spProps', 'npy')

saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_segm-blueNisslRegular_spProps.npy


In [10]:
n_superpixels = len(np.unique(masked_segmentation_relabeled)) - 1

img_superpixelized = mark_boundaries(dm.image, masked_segmentation_relabeled)
img_superpixelized_with_text = img_as_ubyte(img_superpixelized)

dm.save_pipeline_result(img_superpixelized_with_text, 'segmentationWithoutText', 'jpg')

for s in range(n_superpixels):
    cv2.putText(img_superpixelized_with_text, str(s), 
                           tuple(np.floor(sp_centroids[s][::-1]).astype(np.int) - np.array([10,-10])), 
                           cv2.FONT_HERSHEY_DUPLEX,
                           .5, ((255,0,255)), 1)

dm.save_pipeline_result(img_superpixelized_with_text, 'segmentationWithText', 'jpg')

saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_segm-blueNisslRegular_segmentationWithoutText.jpg
saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_segm-blueNisslRegular_segmentationWithText.jpg


In [58]:
emptycanvas_superpixelized = mark_boundaries(np.ones_like(dm.image), masked_segmentation_relabeled, 
                                             color=(0,0,0), outline_color=None)
# emptycanvas_superpixelized = img_as_ubyte(emptycanvas_superpixelized)
alpha_channel = ~ emptycanvas_superpixelized.all(axis=2)
a = np.dstack([emptycanvas_superpixelized, alpha_channel])

dm.save_pipeline_result(a, 'segmentationTransparent', 'png', is_rgb=True)

saved /home/yuncong/project/DavidData2014results/RS139/0009/RS139_x5_0009_segm-blueNisslRegular_segmentationTransparent.png


In [59]:
# display(img_superpixelized_with_text)

In [11]:
from collections import defaultdict

d1 = np.diff(masked_segmentation_relabeled, axis=0).astype(np.bool)
d2 = np.diff(masked_segmentation_relabeled, axis=1).astype(np.bool)
d3 = (masked_segmentation_relabeled[1:,1:]-masked_segmentation_relabeled[:-1,:-1]).astype(np.bool)
d4 = (masked_segmentation_relabeled[1:,:-1]-masked_segmentation_relabeled[:-1,1:]).astype(np.bool)
d5 = (masked_segmentation_relabeled[2:,:]-masked_segmentation_relabeled[:-2,:]).astype(np.bool)
d6 = (masked_segmentation_relabeled[:,2:]-masked_segmentation_relabeled[:,:-2]).astype(np.bool)
d7 = (masked_segmentation_relabeled[2:,2:]-masked_segmentation_relabeled[:-2,:-2]).astype(np.bool)
d8 = (masked_segmentation_relabeled[2:,:-2]-masked_segmentation_relabeled[:-2,2:]).astype(np.bool)

neighbors = [set() for _ in range(n_superpixels)]
edge_coords = defaultdict(set)

def f(d, y1_offset=0, x1_offset=0, y2_offset=0, x2_offset=0):
    ys, xs = np.where(d)
    y1s = ys + y1_offset
    x1s = xs + x1_offset
    y2s = ys + y2_offset
    x2s = xs + x2_offset
    for y1,x1,y2,x2 in zip(y1s,x1s,y2s,x2s):
        l1 = masked_segmentation_relabeled[y1,x1]
        l2 = masked_segmentation_relabeled[y2,x2]
        if l1 != -1:
            neighbors[l1].add(l2)
            edge_coords[frozenset([l1,l2])].add((y1,x1))
            edge_coords[frozenset([l1,l2])].add((y2,x2))
        if l2 != -1:
            try:
                neighbors[l2].add(l1)
            except:
                print l2,l1
            edge_coords[frozenset([l1,l2])].add((y1,x1))
            edge_coords[frozenset([l1,l2])].add((y2,x2))

# cannot work because writing to the same container simultaneous is dangerous
# fargs = [(d1, 0,0,1,0),(d2, 0,0,0,1),(d3, 0,0,1,1),(d4, 0,1,1,0),
#          (d5, 0,0,2,0),(d6, 0,0,0,2),(d7, 0,0,2,2),(d8, 0,2,2,0)]
# Parallel(n_jobs=16)(delayed(f)(t) for t in fargs)
    
f(d1, 0,0,1,0)
f(d2, 0,0,0,1)
f(d3, 0,0,1,1)
f(d4, 0,1,1,0)
f(d5, 0,0,2,0)
f(d6, 0,0,0,2)
f(d7, 0,0,2,2)
f(d8, 0,2,2,0)

# check symmetry; note that this CANNOT check if neighbors is complete
A = np.zeros((n_superpixels, n_superpixels))
for i, nbrs in enumerate(neighbors):
    q = list([j for j in nbrs if j != -1])
    A[i, q] = 1    
assert np.all(A == A.T), 'neighbor list is not symmetric'

for i in range(n_superpixels):
    neighbors[i] -= {i}
    
dm.save_pipeline_result(neighbors, 'neighbors', 'pkl')

edges = edge_coords.keys()

edge_map = -1 * np.ones_like(dm.image, np.int)
for ei, (e, pts_set) in enumerate(edge_coords.iteritems()):
    rs, cs = zip(*pts_set)
    edge_map[np.array(rs), np.array(cs)] = ei
    
def f(edge_coords):
    return [set(zip(*np.hstack([np.mgrid[max(r-5,0):min(r+5, dm.image_height), 
                                         max(c-5,0):min(c+5, dm.image_width)].reshape((2,-1)) 
                                for r,c in pts]).reshape((2,-1)).tolist())) 
            for e, pts in edge_coords]

b = Parallel(n_jobs=16)(delayed(f)(s) for s in np.array_split(edge_coords.items(), 16))
neighborhood_pts = [p for bb in b for p in bb]

edge_neighbors = defaultdict(set)
for ei, pts in enumerate(neighborhood_pts):
    rs, cs = zip(*pts)
    eis = set(edge_map[rs,cs]) - {-1, ei}
    edge_neighbors[edges[ei]] |= set([edges[ei] for ei in eis])

dm.save_pipeline_result(edge_neighbors, 'edgeNeighbors','pkl')

edge_coords_sorted = {}
for e, pts in edge_coords.iteritems():
    X = np.array(list(pts), dtype=np.float) # 2-by-n
    Xc = X - X.mean(axis=0)
    U,S,V = np.linalg.svd(np.dot(Xc.T, Xc))
    u1 = U[:,0]
    projs = np.dot(Xc,u1)
    order = projs.argsort()
    if Xc[order[0],0] > Xc[order[-1],0]:
        order = order[::-1]
    edge_coords_sorted[e] = X[order].astype(np.int)

dm.save_pipeline_result(edge_coords_sorted, 'edgeCoords', 'pkl')

border_len_thresh = 20
neighbors_long_border = [set([nbr for nbr in nbrs if len(edge_coords[frozenset([n,nbr])]) > border_len_thresh]) 
                             for n, nbrs in enumerate(neighbors)]
dm.save_pipeline_result(neighbors_long_border, 'neighborsLongBorder', 'pkl')

saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_segm-blueNisslRegular_neighbors.pkl
saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_edgeNeighbors.pkl
saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_edgeCoords.pkl
saved /home/yuncong/project/DavidData2014results/RS141/0004/RS141_x5_0004_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_neighborsLongBorder.pkl


In [ ]:
# # Compute neighbor lists and connectivity matrix

# from collections import defaultdict
# from itertools import combinations
# from skimage.segmentation import find_boundaries

# # from skimage.morphology import disk
# # from skimage.filter.rank import gradient
    
# # try:
# #     neighbors = dm.load_pipeline_result('neighbors', 'pkl')

# # except:

# q = masked_segmentation_relabeled.astype(np.uint8)
# d1 = np.diff(q, axis=0).astype(np.bool)
# d2 = np.diff(q, axis=0).astype(np.bool)
# d3 = (q[1:,1:]-q[:-1,:-1]).astype(np.bool)
# d4 = (q[1:,:-1]-q[:-1,1:]).astype(np.bool)

# edge_map = np.zeros_like(masked_segmentation_relabeled, np.bool)
# # edge_map = gradient(masked_segmentation_relabeled.astype(np.uint8), square(3))
# # edge_map = find_boundaries(masked_segmentation_relabeled) # behavior not clear
# neighbors = [set() for _ in range(n_superpixels)]

# edge_coords = defaultdict(set)

# adj_size = 2

# for y,x in zip(*np.nonzero(edge_map)):
#     l = masked_segmentation_relabeled[y,x]
#     if l != -1:
#         adjacent_labels = set(masked_segmentation_relabeled[max(0, y-adj_size):min(dm.image_height, y+adj_size),
#                                                         max(0, x-adj_size):min(dm.image_width, x+adj_size)].ravel())
#         neighbors[l] |= adjacent_labels

#         for al in adjacent_labels - {l}:
#             edge_coords[frozenset([l,al])].add((y,x))

# for i in range(n_superpixels):
#     neighbors[i] -= {i}

# # dm.save_pipeline_result(neighbors, 'neighbors', 'pkl')

# # dm.save_pipeline_result(edge_coords, 'edgeCoords', 'pkl')

# # border_len_thresh = 20
# # neighbors_long_border = [set([nbr for nbr in nbrs if len(edge_coords[frozenset([n,nbr])]) > border_len_thresh]) 
# #                              for n, nbrs in enumerate(neighbors)]
# # dm.save_pipeline_result(neighbors_long_border, 'neighborsLongBorder', 'pkl')


# # neighbors_dict = dict(zip(np.arange(n_superpixels), [list(i) for i in neighbors]))
# # neighbor_graph = from_dict_of_lists(neighbors_dict)